In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from output import output

import pandas as pd
import time
import wget
import os
import glob

In [2]:
out_path = f'{os.getcwd()}/output/{output.get()}'
df = pd.read_csv(f'{out_path}/out_submit.csv')

In [3]:
items = [{'Index': x['Index'], 'Link': x['Link'], 'Energy': 'NaN'} for i, x in df.iterrows()]

In [4]:
driver = webdriver.Chrome()

In [5]:
for i, item in enumerate(items):
    driver.get(item['Link'])
    time.sleep(3)

    print('Collecting data...', item['Link'])
    element = driver.find_element(By.XPATH, '/html/body/center/table[3]/tbody/tr[2]/td[1]')
    item['Energy'] = element.text
    # Docking Score (1st model)

    dock_path = f'{out_path}/docking/pep{item["Index"]}'
    if not os.path.exists(dock_path):
        os.makedirs(dock_path)
    # Create docking results folder

    top10_models = f'{item["Link"]}top10_models.tar.gz'
    wget.download(top10_models)
    os.system(f'tar -xzf top10_models.tar.gz -C {dock_path}')
    # Download and extract files containing the 10 best positions of each peptide

    [directory] = os.listdir(dock_path)
    topmodels = os.listdir(f'{dock_path}/{directory}')
    for model in topmodels:
        os.rename(f'{dock_path}/{directory}/{model}', f'{dock_path}/{model}')
    os.rmdir(f'{dock_path}/{directory}')
    os.remove('top10_models.tar.gz')
    # Delete file and move models to pep folder

    print(f'Pep{item["Index"]} docking has completed. See top 10 models for it in:\n  {dock_path}\n')

100% [..............................................................................] 21581 / 21581Pep5 docking has completed. See top 10 models for it in:
  D:\WilliamJSS\Projects\LBQC\BioPep/output/2022.11.02-08.25.48-Task/docking/pep5

100% [..............................................................................] 21651 / 21651Pep6 docking has completed. See top 10 models for it in:
  D:\WilliamJSS\Projects\LBQC\BioPep/output/2022.11.02-08.25.48-Task/docking/pep6

100% [..............................................................................] 14717 / 14717Pep11 docking has completed. See top 10 models for it in:
  D:\WilliamJSS\Projects\LBQC\BioPep/output/2022.11.02-08.25.48-Task/docking/pep11

100% [..............................................................................] 15101 / 15101Pep12 docking has completed. See top 10 models for it in:
  D:\WilliamJSS\Projects\LBQC\BioPep/output/2022.11.02-08.25.48-Task/docking/pep12

100% [..................................

In [6]:
driver.close()

In [7]:
df['Energies'] = [item['Energy'] for item in items]
df.to_csv(f'{out_path}/results_dock.csv')